In [39]:
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm

# **File Name Convention**
## **Gen_X_Gen_Y_pgini_value1pmax_value2**
## X-outage Generator
## Y-Generator whose pgini is varied,here pgini is the dispatched power

In [3]:
df = pd.read_csv(r"/home/rakin/New_data/Gen_1_Gen_1pgini_800pmax_8500.csv")
df.head(2)

,Time,Frequency,Spinning Reserve,Inertia,Active Power Rating,Dispatched Power,ROCOF,Nadir,Settling Frequency
0,-0.10,50.000000,8391.649633,10.769797,8500.0,800.0,0.360404708256,46.4205017945,48.8687801361
1,-0.05,49.999345,8394.857276,10.769797,595.0,0.0,,,


## **Target**
1. Input Features ---> dipatched power of all generator (before disturbance), Reserve Size, disturbance size
2. Output------------> Nadir, ROCOF, Settling Frequency

In [56]:
def read_data(df,gen_num):
    # time = df['Time']
    # frequency = df['Frequency']
    # inertia = df['inertia']
    Pmax = float(df['Active Power Rating'][gen_num-1])
    P_dispatched = df['Dispatched Power'][0:10]
    disturbance = float(P_dispatched[gen_num-1])
    reserve_size = Pmax - disturbance
    ROCOF = float(df['ROCOF'][0])
    Nadir = float(df['Nadir'][0])
    Settling_frequency = float(df['Settling Frequency'][0])

    return    P_dispatched.astype(float), reserve_size, disturbance,ROCOF, Settling_frequency, Nadir# add ROCOF, Settling Frequency, Nadir

    

In [89]:
filename = '/home/rakin/New_data/Gen_1_Gen_1pgini_800pmax_8500.csv'
# array = re.split("_",filename)
# print(array[2])
pow = pd.read_csv(filename)
P_dispatched, reserve_size, disturbance,ROCOF, Settling_frequency, Nadir = read_data(pow,1)
P_dispatched[0:10]

0    800.0
1      0.0
2    650.0
3    632.0
4    254.0
5    650.0
6    560.0
7    540.0
8    830.0
9    250.0
Name: Dispatched Power, dtype: float64

In [52]:
columns =  ["gen_{}_dispatched".format(i+1) for i in range(10)]
columns.extend(['Reserve Size', 'Disturbance Size','ROCOF', 'Settling Frequency', 'Nadir'])

In [99]:
final_dataframe = pd.DataFrame(columns=columns)
base_dir = '/home/rakin/New_data/'
def pre_processing(base_dir, final_dataframe):
    files = os.listdir(base_dir)
    i = 0
    for file in tqdm(files):
        gen_num = int(re.split("_",file)[1])
        df = pd.read_csv(base_dir+file)
        # temp1 = []
        P_dispatched, reserve_size, disturbance,ROCOF, Settling_frequency, Nadir = read_data(df,gen_num)
        # for j in range(10):
        #     final_dataframe.iloc[i,j]=P_dispatched[j]
        P_dispatched = list(P_dispatched)
        temp2 = [reserve_size, disturbance,ROCOF, Settling_frequency, Nadir]
        P_dispatched.extend(temp2)
        final_dataframe.loc[i,:]= P_dispatched
        i = i+1


In [100]:
pre_processing(base_dir, final_dataframe)

100%|██████████| 1995/1995 [00:19<00:00, 100.40it/s]


dtype('O')